# 타이타닉 
---
- 타이타닉에 탑승한 사람들의 신상정보를 활용하여, 승선한 사람들의 생존여부를 예측하는 모델을 생성
- 여러 시각화 도구(matplotlib, seaborn, plotly), 데이터 분석 도구(pandas, numpy), 머신 러닝 도구(sklearn)을 사용


## 일단 사용하는 모듈 
 - 넘파이 
 - 판다스 
 - 맷폴랫
 - 시본 
 
 - missingno =>  missingno 패키지를 사용하면 누락데이터에 대한 시각화
 - 워닝 방지 


In [4]:
# !pip install missingno 

In [2]:
# import
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)
# matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 
# 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편합니다.

import missingno as msno


# ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 앞으로의 프로세스 
### 1. 데이터 셋 확인 
    - null data 확인하고 추후 수정 
### 2. 탐색적 데이터 분석(exploratory data analysis)  
    - 여러 특징(feature)들을 개별적으로 분석하고 특징들의 상관관계를 확인 
    - 여러 시각화 툴을 이용하여 통찰력(insight)을 얻음 
### 3. 특징을 엔지니어링한다 
    - 모델의 성능을 높일 수 있도록 feature engineering을한다 
    - one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리등...
### 4. 모델 만들기 
    - 사잇킷런을 사용하여 모델을 만듬 
    - sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용
### 5. 모델 학습 및 예측 
    - 트레인 셋을 가지고 모델을 학습
    - 테스트 셋으로 예측을 (prediction)함 
### 6. 모델 평가 
    - 예측 성능이 원하는 수준인지 판단 
    - 풀려는 문제에 따라 모델을 평가하는 방식이 달라짐 

## 1. 데이터 셋 확인
- null data 확인하고 추후 수정 
- 판다스 => 테이블화 된 데이터를 다루는데 가장 최적화 된 라이브러리 
- pandas를 사용하여 데이터 셋의 간단한 통계적 분석, 복잡한 처리들을 간단한 메소드를 사용하여 작업  
- 판다스는 반복또 반복
---
- 캐글의 데이터 셋은 train, testset로 나누어짐 

In [4]:
df_train = pd.read_csv('../input/train.csv')
df_test  = pd.read_csv('../input/test.csv')

In [6]:
df_train.head()
# print(df_train.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 제공 데이터 확인 
- 데이터의 특징은 class, Age, SibSp, Parch, Fare
- 예측하고자 하는것은 생존자 
    => target label 은 Survived 
- parch => 함께 탑승한 부모, 아이의 수 (정량적)
- SibSp => 함께 탑승한 형제와 배우자의 수 (정략적)
- Fare => 탑승료 

---
- describe() 메소드=> 특징을 가진 통계치들을 반환     
[describe()_Eng](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)

In [8]:
# describe()메소드 사용 
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
